In [36]:
from pathlib import Path
import pandas as pd
import duckdb

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
path_DB = data_root / "건축물대장_2025년_02월.db"

results_dir = Path("../results")

In [37]:
# Open a DuckDB connection
con = duckdb.connect(database=path_DB, read_only=True)

# print the list of tables in the database
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])
    print()

# show heads of the tables
for table in tables:
    table_name = table[0]
    print(f"Head of {table_name}:")
    # print(con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf())
    print(con.sql(f"SELECT * FROM {table_name} LIMIT 5"))
    print()

Tables in the database:
기본개요

층별개요

표제부

Head of 기본개요:
┌────────────────────┬─────────────────────────┬────────────────┬───────────────────┬────────────────┬───────────────────┬───────────────────────────────────┬─────────────────────────────────────────┬─────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────────┬────────────────────┬──────────────────────┬──────────────┬──────────────┬───────────┬───────────┬───────────┬──────────────┬──────────────┬──────────────┬───────────┐
│ 관리_건축물대장_PK │ 관리_상위_건축물대장_PK │ 대장_구분_코드 │ 대장_구분_코드_명 │ 대장_종류_코드 │ 대장_종류_코드_명 │             대지_위치             │            도로명_대지_위치             │ 건물_명 │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 외필지_수 │ 새주소_도로_코드 │ 새주소_법정동_코드 │ 새주소_지상지하_코드 │ 새주소_본_번 │ 새주소_부_번 │ 지역_코드 │ 지구_코드 │ 구역_코드 │ 지역_코드_명 │ 지구_코드_명 │ 구역_코드_명 │ 생성_일자 │
│      varchar       │         varchar         │    varchar     │      varchar      

In [38]:
# Create a list to store the table names and their record counts
table_counts = []

# Iterate through the tables and count the records
for table in tables:
    table_name = table[0]
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    table_counts.append({"Table Name": table_name, "Record Count": count})

# Convert the list to a DataFrame
record_counts_df = pd.DataFrame(table_counts)

# Display the DataFrame
display(record_counts_df)

,Table Name,Record Count
0,기본개요,27994157
1,층별개요,21048715
2,표제부,8027067


In [39]:
# Create a temporary view with the selected columns
표제부_필터링 = con.sql("""
    SELECT 
        "주_용도_코드",
        "주_용도_코드_명",
        "기타_용도"
    FROM 표제부
    WHERE "기타_용도" IS NOT NULL AND "기타_용도" != ''
        AND "주_용도_코드_명" IS NOT NULL AND "주_용도_코드_명" != ''
""")
# Count the records in the filtered view
con.sql("SELECT COUNT(*) FROM 표제부_필터링").fetchone()[0]

7969401

In [40]:
# Perform value count using SQL
value_counts_sql = con.sql("""
    SELECT 
        "주_용도_코드",
        "주_용도_코드_명",
        "기타_용도",
        COUNT(*) AS "Count"
    FROM 표제부_필터링
    GROUP BY "주_용도_코드", "주_용도_코드_명", "기타_용도"
    ORDER BY "Count" DESC
""")

# Fetch and display the result
value_counts_df = value_counts_sql.fetchdf()
display(value_counts_df.head(5))
# Save the top 30 rows to a CSV file
# value_counts_df.head(30).to_csv(
#     results_dir / "표제부_용도_기재내용_top30.csv", index=False, encoding="utf-8-sig"
# )

,주_용도_코드,주_용도_코드_명,기타_용도,Count
0,01000,단독주택,단독주택,1722659
1,01000,단독주택,주택,1592478
2,04000,제2종근린생활시설,제2종근린생활시설,327131
3,17000,공장,공장,304174
4,18000,창고시설,창고시설,230618


In [41]:
# Create a temporary view to split the '기타_용도' column into arrays using a regular expression
value_counts_split = con.sql("""
    SELECT 
        *,
        regexp_split_to_array(lower("기타_용도"), '[^\\p{L}\\d]+') AS "기타_용도_분리"
    FROM value_counts_sql
""")

# Fetch and display the result
value_counts_split.fetchdf()

,주_용도_코드,주_용도_코드_명,기타_용도,Count,기타_용도_분리
0,01000,단독주택,단독주택,1722659,[단독주택]
1,01000,단독주택,주택,1592478,[주택]
2,04000,제2종근린생활시설,제2종근린생활시설,327131,[제2종근린생활시설]
3,17000,공장,공장,304174,[공장]
4,18000,창고시설,창고시설,230618,[창고시설]
...,...,...,...,...,...
261450,01000,단독주택,"사무실, 주택, 소매점",1,"[사무실, 주택, 소매점]"
261451,01000,단독주택,단독주택[서재],1,"[단독주택, 서재, ]"
261452,17000,공장,공장(크링카바켓엘리베이터),1,"[공장, 크링카바켓엘리베이터, ]"
261453,14000,업무시설,"사무실, 근린생활시설, 위험물저장및처리시설",1,"[사무실, 근린생활시설, 위험물저장및처리시설]"


In [42]:
# Create a temporary view to split the '기타_용도' column into rows using a regular expression
value_counts_split_table = con.sql("""
    SELECT 
        *,
        regexp_split_to_table(lower("기타_용도"), '[^\\p{L}\\d]+') AS "기타_용도_분리"
    FROM value_counts_sql
""")

# Fetch and display the result
value_counts_split_table.fetchdf()

,주_용도_코드,주_용도_코드_명,기타_용도,Count,기타_용도_분리
0,01000,단독주택,단독주택,1722659,단독주택
1,01000,단독주택,주택,1592478,주택
2,04000,제2종근린생활시설,제2종근린생활시설,327131,제2종근린생활시설
3,17000,공장,공장,304174,공장
4,18000,창고시설,창고시설,230618,창고시설
...,...,...,...,...,...
834252,14000,업무시설,"업무시설, 제1종근린생활시설, 공동주택",1,공동주택
834253,02000,공동주택,"지하주차장,통신실,제연휀룸,재활용보관소",1,지하주차장
834254,02000,공동주택,"지하주차장,통신실,제연휀룸,재활용보관소",1,통신실
834255,02000,공동주택,"지하주차장,통신실,제연휀룸,재활용보관소",1,제연휀룸


In [43]:
# Group by valid (5 digit) '주_용도_코드',
# sum the 'Count' column, and filter where the sum is >= 100
class_counts_sql = con.sql("""
    SELECT
        "주_용도_코드",
        "주_용도_코드_명",
        SUM("Count") AS "Total_Count"
    FROM value_counts_sql
    WHERE LENGTH("주_용도_코드") = 5 AND "주_용도_코드" ~ '^\\d+$'
    GROUP BY "주_용도_코드", "주_용도_코드_명"
    HAVING SUM("Count") >= 100
    ORDER BY "Total_Count" DESC
""")

# Fetch and display the result
class_counts_df = class_counts_sql.fetchdf().astype({"Total_Count": int})
display(class_counts_df)
# Save to a CSV file
class_counts_df.to_csv(
    results_dir / "naive_bayes_클래스별_문서빈도.csv",
    index=False,
    encoding="utf-8-sig",
)

,주_용도_코드,주_용도_코드_명,Total_Count
0,01000,단독주택,4326700
1,04000,제2종근린생활시설,718151
2,03000,제1종근린생활시설,594056
3,02000,공동주택,578273
4,21000,동물및식물관련시설,439989
5,18000,창고시설,409787
6,17000,공장,404164
7,10000,교육연구시설,88934
8,11000,노유자시설,48184
9,15000,숙박시설,47324


In [44]:
# Group by '기타_용도_분리',
# sum the 'Count' column, and filter where the sum is >= 100
word_counts_sql = con.sql("""
    SELECT
        "기타_용도_분리",
        SUM("Count") AS "Total_Count"
    FROM value_counts_split_table
    WHERE "기타_용도_분리" IS NOT NULL AND "기타_용도_분리" != ''
    GROUP BY "기타_용도_분리"
    HAVING SUM("Count") >= 100
    ORDER BY "Total_Count" DESC
""")

# Fetch and display the result
word_counts_df = word_counts_sql.fetchdf().astype({"Total_Count": int})
display(word_counts_df)
# Save to a CSV file
word_counts_df.to_csv(
    results_dir / "naive_bayes_단어별_문서빈도.csv",
    index=False,
    encoding="utf-8-sig",
)

,기타_용도_분리,Total_Count
0,단독주택,2114386
1,주택,2067671
2,근린생활시설,513269
3,제2종근린생활시설,503960
4,공장,362449
...,...,...
1067,마사,100
1068,기전실,100
1069,욕탕,100
1070,포,100


In [45]:
# Construct training data with '주_용도_코드' and '기타_용도_분리'
# Filter with class_counts_sql and word_counts_sql
train_data_sql = con.sql("""
    SELECT
        t."주_용도_코드" AS class,
        t."기타_용도_분리" AS word,
        t."Count" AS weight,
    FROM value_counts_split_table t
    JOIN class_counts_sql c
        ON t."주_용도_코드" = c."주_용도_코드"
    JOIN word_counts_sql w
        ON t."기타_용도_분리" = w."기타_용도_분리"
    WHERE t.Count >= 100
    ORDER BY t."Count" DESC
""")
train_df = train_data_sql.fetchdf()
train_df

,class,word,weight
0,01000,단독주택,1722659
1,01000,주택,1592478
2,04000,제2종근린생활시설,327131
3,17000,공장,304174
4,18000,창고시설,230618
...,...,...,...
3919,04000,단독주택,100
3920,04000,2종근린생활시설,100
3921,17000,공장,100
3922,17000,전기실,100


In [46]:
import duckdb
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


# 2. Vectorize & train

# The multinomial Naive Bayes classifier is suitable for classification with
# discrete features (e.g., word counts for text classification). The multinomial
# distribution normally requires integer feature counts. However, in practice,
# fractional counts such as tf-idf may also work.

# word is already a word, but no harm from count vectorization
# regex does not support unicode, so we need to customize the pattern
vectorizer = CountVectorizer(token_pattern=r"[\\w\\d가-힣]+")
X = vectorizer.fit_transform(train_df["word"])
y = train_df["class"]
weights = train_df["weight"]
clf = MultinomialNB(alpha=1.0)
clf.fit(X, y, sample_weight=weights)

MultinomialNB()

In [47]:
vectorizer.vocabulary_

{'단독주택': 153,
 '주택': 405,
 '제': 372,
 '종근린생활시설': 385,
 '공장': 36,
 '창고시설': 434,
 '동': 174,
 '식물관련시설': 275,
 '공동주택': 34,
 '다세대주택': 147,
 '근린생활시설': 76,
 '창고': 429,
 '아파트': 282,
 '동물및식물관련시설': 178,
 '농가주택': 106,
 '축사': 443,
 '교육연구시설': 63,
 '다가구주택': 142,
 '동물관련시설': 176,
 '점포': 366,
 '숙박시설': 268,
 '변소': 212,
 '노유자시설': 97,
 '자동차관련시설': 342,
 '위험물저장및처리시설': 324,
 '경비실': 19,
 '종교시설': 382,
 '연립주택': 296,
 '교육연구및복지시설': 62,
 '교정및군사시설': 67,
 '소매점': 263,
 '지하주차장': 424,
 '업무시설': 290,
 '다가구용단독주택': 140,
 '가구': 1,
 '일반음식점': 338,
 '동식물관련시설': 180,
 '사무소': 247,
 '자원순환관련시설': 343,
 '제조업소': 379,
 '운동시설': 315,
 '문화및집회시설': 197,
 '퇴비사': 460,
 '근린공공시설': 74,
 '다가구': 137,
 '화장실': 480,
 '관광휴게시설': 47,
 '사무실': 248,
 '동물': 175,
 '및': 204,
 '야영장시설': 284,
 '마을회관': 186,
 '농업용창고': 133,
 '일반공장': 335,
 '물치': 198,
 '농어가주택': 125,
 '부속창고': 235,
 '주거시설': 396,
 '부속': 231,
 '외': 308,
 '다세대': 146,
 '버섯재배사': 210,
 '영업소': 301,
 '분뇨': 238,
 '쓰레기처리시설': 279,
 '판매시설': 463,
 '발전시설': 207,
 '국방군사시설': 72,
 '관리사': 52,
 '세대': 260,
 '의료시설': 333,
 '

In [48]:
# test the classifier
test_text = "도시형생활주택(단지형다세대주택)다세대주택(8세대)외 1"
X_test = vectorizer.transform([test_text])
X_test.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [49]:
clf.predict(X_test)[0]

np.str_('02000')

In [50]:
# 3. Register a prediction UDF
def nb_predict(text: str) -> str:
    x = vectorizer.transform([text])
    return clf.predict(x)[0]


# Remove the function if it already exists
try:
    con.remove_function("nb_predict")
except Exception as e:
    if "No function by the name of" not in str(e):
        raise

# Register the function

con.create_function("nb_predict", nb_predict)

In [51]:
preds = con.sql("""
  SELECT
    *,
    nb_predict("기타_용도") AS predicted_label,
    CASE 
      WHEN "주_용도_코드" = predicted_label THEN 1 
      ELSE 0 
    END AS is_correct
  FROM value_counts_sql
""")
preds_label = con.sql("""
  SELECT
    preds.*,
    class_counts_sql."주_용도_코드_명" AS "predicted_label_name",
  FROM preds
  LEFT JOIN class_counts_sql
    ON preds."predicted_label" = class_counts_sql."주_용도_코드"
""")
preds_df = preds_label.fetchdf()
preds_df

,주_용도_코드,주_용도_코드_명,기타_용도,Count,predicted_label,is_correct,predicted_label_name
0,14000,업무시설,"일반업무시설, 교육연구시설, 제2종근린생활시설",1,04000,0,제2종근린생활시설
1,01000,단독주택,"창고,부속사,주택",1,01000,1,단독주택
2,03000,제1종근린생활시설,"화장실,세면장",1,01000,0,단독주택
3,17000,공장,공작물YT-5,1,01000,0,단독주택
4,04000,제2종근린생활시설,"공장(사무실,창고,숙직실)",1,17000,0,공장
...,...,...,...,...,...,...,...
261450,14000,업무시설,"업무시설, 근린생활시설, 주차장",3,14000,1,업무시설
261451,21000,동물및식물관련시설,양돈-퇴비사,3,21000,1,동물및식물관련시설
261452,04000,제2종근린생활시설,"대중음식점, 다방, 사무실, 점포, 주택",3,04000,1,제2종근린생활시설
261453,17000,공장,사무실및공장,3,01000,0,단독주택


In [52]:
# Save preds to a CSV file
preds_df.to_csv(
    results_dir / "naive_bayes_예측결과.csv",
    index=False,
    encoding="utf-8-sig",
)